In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler, StandardScaler, RobustScaler
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import mlflow
#from mlflow import pyspark
from mlflow.tracking import MlflowClient

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'classifyHW'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [3]:
# load the data
fil = '../../data/beatsdataset.csv'
beats = spark.read.format('csv').options(header=True, inferSchema=True).load(fil).drop('_c0')
print('%d records'%beats.count())
display(beats.limit(10).toPandas())
beats.printSchema()

2300 records


,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,10-MFCCs2m,11-MFCCs3m,12-MFCCs4m,13-MFCCs5m,14-MFCCs6m,15-MFCCs7m,16-MFCCs8m,17-MFCCs9m,18-MFCCs10m,19-MFCCs11m,20-MFCCs12m,21-MFCCs13m,22-ChromaVector1m,23-ChromaVector2m,24-ChromaVector3m,25-ChromaVector4m,26-ChromaVector5m,27-ChromaVector6m,28-ChromaVector7m,29-ChromaVector8m,30-ChromaVector9m,31-ChromaVector10m,32-ChromaVector11m,33-ChromaVector12m,34-ChromaDeviationm,35-ZCRstd,36-Energystd,37-EnergyEntropystd,38-SpectralCentroidstd,39-SpectralSpreadstd,40-SpectralEntropystd,41-SpectralFluxstd,42-SpectralRolloffstd,43-MFCCs1std,44-MFCCs2std,45-MFCCs3std,46-MFCCs4std,47-MFCCs5std,48-MFCCs6std,49-MFCCs7std,50-MFCCs8std,51-MFCCs9std,52-MFCCs10std,53-MFCCs11std,54-MFCCs12std,55-MFCCs13std,56-ChromaVector1std,57-ChromaVector2std,58-ChromaVector3std,59-ChromaVector4std,60-ChromaVector5std,61-ChromaVector6std,62-ChromaVector7std,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class
0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,1.594074,0.011276,0.204468,0.042072,0.048552,0.158505,0.118984,-0.147956,-0.186152,-0.026418,-0.007264,-0.017900,0.011581,0.008747,0.041081,0.014497,0.025711,0.012587,0.060170,0.002864,0.004631,0.009576,0.026079,0.004161,0.032185,0.050143,0.047313,0.102995,0.041285,0.017725,0.414831,0.005867,0.133778,0.838302,0.505911,0.356206,0.336074,0.288888,0.278649,0.283437,0.300305,0.287688,0.296692,0.258531,0.238352,0.194701,0.013138,0.011665,0.032049,0.015464,0.020453,0.012943,0.046397,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom
1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,1.261364,-0.113015,0.001718,-0.052682,0.204130,0.153013,0.067214,-0.013227,-0.059440,-0.008604,0.114257,0.171009,0.006535,0.002646,0.086485,0.008391,0.016442,0.009006,0.087948,0.002472,0.006549,0.007412,0.015386,0.005978,0.041116,0.043713,0.043721,0.099449,0.039386,0.018946,0.407164,0.003613,0.110334,0.624185,0.476993,0.353151,0.335550,0.283832,0.269621,0.244150,0.246660,0.257190,0.272036,0.269477,0.222393,0.187471,0.006761,0.003152,0.058923,0.009012,0.016106,0.009386,0.071726,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom
2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,1.425185,0.186749,0.417114,0.076406,0.190803,-0.016302,0.075038,0.107870,0.216874,0.095604,0.020977,-0.037011,0.007143,0.002960,0.220526,0.005639,0.010151,0.007453,0.043907,0.001240,0.004347,0.007989,0.017622,0.002636,0.066049,0.032920,0.037618,0.117704,0.041509,0.022645,0.340130,0.007697,0.085784,1.028740,0.449133,0.297935,0.266731,0.258299,0.275012,0.218368,0.198390,0.210177,0.212533,0.204458,0.197634,0.164910,0.007836,0.003079,0.093865,0.005692,0.008212,0.005451,0.042900,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom
3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,1.463686,0.226548,0.404531,0.117699,0.081861,0.053974,0.164865,0.014919,0.117090,0.027778,-0.063173,-0.052606,0.010724,0.003340,0.125459,0.005728,0.014695,0.006322,0.072154,0.001628,0.003493,0.011463,0.032204,0.004738,0.046159,0.036349,0.061960,0.134908,0.032564,0.020036,0.365068,0.005215,0.086336,0.769981,0.425496,0.245312,0.260132,0.224220,0.207597,0.199472,0.207818,0.189912,0.185509,0.187273,0.177629,0.164740,0.008330,0.003528,0.061426,0.005443,0.012382,0.004985,0.057999,0.001591,0.003514,0.009477,0.023162,0.004165,0.015379,133.333333,0.158876,129.0,BigRoom
4,0.151730,0.148405,3.194498,0.293730,0.267231,1.353005,0.003872,0.292055,-21.371157,1.187854,0.184415,0.363724,0.232119,0.112277,0.107335,0.159296,0.067213,-0.018713,0.091529,0.117344,0.091616,0.009624,0.004031,0.076133,0.008175,0.016267,0.009927

root
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (nullable = true)
 |-- 23-ChromaVector2m:

In [4]:
# specify the response colum & tabulate by it
responseVar = 'class'
display(beats.groupBy(responseVar).count().toPandas())

,class,count
0,PsyTrance,100
1,HardDance,100
2,Breaks,100
3,HardcoreHardTechno,100
4,IndieDanceNuDisco,100
5,Trance,100
6,DeepHouse,100
7,ElectronicaDowntempo,100
8,ReggaeDub,100
9,Minimal,100


In [5]:
# check for missing values
cnt = beats.count()
nullCounts = {colm:beats.select(colm).where(col(colm).isNull()).count() for colm in beats.columns}
nullCounts = {colm:(ncnt, ncnt/cnt) for (colm, ncnt) in nullCounts.items()}

# pretty print
nullCountsDF = pd.DataFrame(nullCounts).T.reset_index(drop=False).sort_values(1, ascending=False)
nullCountsDF.columns = ['Column', 'Freq.', 'Rel. Freq.']
nullCountsDF = nullCountsDF.merge(pd.DataFrame([[colm.name, colm.dataType] for colm in beats.schema], columns=['Column', 'Type']),
                                how='inner', on=['Column'])
display(nullCountsDF)

,Column,Freq.,Rel. Freq.,Type
0,1-ZCRm,0.0,0.0,DoubleType
1,2-Energym,0.0,0.0,DoubleType
2,53-MFCCs11std,0.0,0.0,DoubleType
3,52-MFCCs10std,0.0,0.0,DoubleType
4,51-MFCCs9std,0.0,0.0,DoubleType
5,50-MFCCs8std,0.0,0.0,DoubleType
6,49-MFCCs7std,0.0,0.0,DoubleType
7,48-MFCCs6std,0.0,0.0,DoubleType
8,47-MFCCs5std,0.0,0.0,DoubleType
9,46-MFCCs4std,0.0,0.0,DoubleType


In [7]:
''' prep the data for modeling '''
# set inputs
inpColumns = beats.columns[:-1]

# create the features vector
assr = VectorAssembler(inputCols=inpColumns, outputCol='features_orig')
beatsML = assr.transform(beats)

# scale the features
scalr = RobustScaler(inputCol='features_orig', outputCol='features')
beatsML = scalr.fit(beatsML).transform(beatsML)

# make the response numerical
indxr = StringIndexer(inputCol=responseVar, outputCol='label')
beatsML = indxr.fit(beatsML).transform(beatsML)

# talk
display(beatsML.limit(10).toPandas())
beatsML.select('features', 'label').show(truncate=True)
beatsML.select('features').take(1)
print('First row features = %s'%beatsML.select('features').take(1)[0])

,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,10-MFCCs2m,11-MFCCs3m,12-MFCCs4m,13-MFCCs5m,14-MFCCs6m,15-MFCCs7m,16-MFCCs8m,17-MFCCs9m,18-MFCCs10m,19-MFCCs11m,20-MFCCs12m,21-MFCCs13m,22-ChromaVector1m,23-ChromaVector2m,24-ChromaVector3m,25-ChromaVector4m,26-ChromaVector5m,27-ChromaVector6m,28-ChromaVector7m,29-ChromaVector8m,30-ChromaVector9m,31-ChromaVector10m,32-ChromaVector11m,33-ChromaVector12m,34-ChromaDeviationm,35-ZCRstd,36-Energystd,37-EnergyEntropystd,38-SpectralCentroidstd,39-SpectralSpreadstd,40-SpectralEntropystd,41-SpectralFluxstd,42-SpectralRolloffstd,43-MFCCs1std,44-MFCCs2std,45-MFCCs3std,46-MFCCs4std,47-MFCCs5std,48-MFCCs6std,49-MFCCs7std,50-MFCCs8std,51-MFCCs9std,52-MFCCs10std,53-MFCCs11std,54-MFCCs12std,55-MFCCs13std,56-ChromaVector1std,57-ChromaVector2std,58-ChromaVector3std,59-ChromaVector4std,60-ChromaVector5std,61-ChromaVector6std,62-ChromaVector7std,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class,features_orig,features,label
0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,1.594074,0.011276,0.204468,0.042072,0.048552,0.158505,0.118984,-0.147956,-0.186152,-0.026418,-0.007264,-0.017900,0.011581,0.008747,0.041081,0.014497,0.025711,0.012587,0.060170,0.002864,0.004631,0.009576,0.026079,0.004161,0.032185,0.050143,0.047313,0.102995,0.041285,0.017725,0.414831,0.005867,0.133778,0.838302,0.505911,0.356206,0.336074,0.288888,0.278649,0.283437,0.300305,0.287688,0.296692,0.258531,0.238352,0.194701,0.013138,0.011665,0.032049,0.015464,0.020453,0.012943,0.046397,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom,"[0.136439587512, 0.0888612604609, 3.2012005559...","[2.621763736423596, 1.70103481603904, 32.56325...",0.0
1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,1.261364,-0.113015,0.001718,-0.052682,0.204130,0.153013,0.067214,-0.013227,-0.059440,-0.008604,0.114257,0.171009,0.006535,0.002646,0.086485,0.008391,0.016442,0.009006,0.087948,0.002472,0.006549,0.007412,0.015386,0.005978,0.041116,0.043713,0.043721,0.099449,0.039386,0.018946,0.407164,0.003613,0.110334,0.624185,0.476993,0.353151,0.335550,0.283832,0.269621,0.244150,0.246660,0.257190,0.272036,0.269477,0.222393,0.187471,0.006761,0.003152,0.058923,0.009012,0.016106,0.009386,0.071726,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom,"[0.117038518483, 0.108389033282, 3.19400106287...","[2.2489612371225074, 2.074846995565889, 32.490...",0.0
2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,1.425185,0.186749,0.417114,0.076406,0.190803,-0.016302,0.075038,0.107870,0.216874,0.095604,0.020977,-0.037011,0.007143,0.002960,0.220526,0.005639,0.010151,0.007453,0.043907,0.001240,0.004347,0.007989,0.017622,0.002636,0.066049,0.032920,0.037618,0.117704,0.041509,0.022645,0.340130,0.007697,0.085784,1.028740,0.449133,0.297935,0.266731,0.258299,0.275012,0.218368,0.198390,0.210177,0.212533,0.204458,0.197634,0.164910,0.007836,0.003079,0.093865,0.005692,0.008212,0.005451,0.042900,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom,"[0.0853077737447, 0.128525418596, 3.1238373468...","[1.6392370551487672, 2.4603095954731096, 31.77...",0.0
3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,1.463686,0.226548,0.404531,0.117699,0.081861,0.053974,0.164865,0.014919,0.117090,0.027778,-0.063173,-0.052606,0.010724,0.003340,0.125459,0.005728,0.014695,0.006322,0.072154,0.001628,0.003493,0.011463,0.032204,0.004738,0.046159,0.036349,0.061960,0.134908,0.032564,0.020036,0.365068,0.005215,0.086336,0.769981,0.425496,0.245312,0.260132,0.224220,0.207597,0.199472,0.207818,0.189912,0.185509,0.187273,0.177629,0.164740,0.008330,0.003528,0.061426,0.005443,0.012382,

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2.62176373642359...|  0.0|
|[2.24896123712250...|  0.0|
|[1.63923705514876...|  0.0|
|[1.98016236288167...|  0.0|
|[2.91557666352305...|  0.0|
|[2.44127481235333...|  0.0|
|[2.37111042283127...|  0.0|
|[2.69070524221297...|  0.0|
|[2.26042682391396...|  0.0|
|[2.64022209222507...|  0.0|
|[2.86002034317062...|  0.0|
|[2.30292299045409...|  0.0|
|[1.51190686506125...|  0.0|
|[2.65818792948456...|  0.0|
|[1.94661756778893...|  0.0|
|[2.55302183832814...|  0.0|
|[2.9458137836587,...|  0.0|
|[2.26746739169909...|  0.0|
|[2.13278345629527...|  0.0|
|[2.38849043835619...|  0.0|
+--------------------+-----+
only showing top 20 rows

First row features = [2.621763736423596,1.70103481603904,32.56325622817311,4.349129196001776,9.769208712661392,2.2303314225910627,1.0969734439957743,2.284572126925484,-13.74254910397548,2.089421920313096,0.029811040877083257,0.8793220327583383,0.23157344027726573,0.298682758833

In [8]:
''' split for cross-val '''
trainPerc = 0.7
randSeed = 42
tranBeats, testBeats = beatsML.randomSplit([trainPerc, 1.0 - trainPerc], seed=randSeed)

### Logistic Regression

In [ ]:
''' now use the builtin cross-validator '''
# define the evaulation function
acc = MulticlassClassificationEvaluator(metricName='accuracy')
#estimator
logreg = LogisticRegression()
# parameters grid
params = (ParamGridBuilder().addGrid(logreg.threshold, [0.4, 0.5, 0.6]).addGrid(logreg.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
# cross-validator
cv = CrossValidator(estimator=logreg, estimatorParamMaps=params, evaluator=acc, numFolds=5)

# run
fitModel = cv.fit(trainASD.select('features', 'label'))

In [ ]:
bestModel = fitModel.bestModel

# best model parameters
print('Threshold = %0.3f, Elastic Net = %0.3f'%(bestModel.getThreshold(), bestModel.getElasticNetParam()))

# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)

In [ ]:
# only available for logistic regression
summ = bestModel.summary
summ.predictions.describe().show()
summ.objectiveHistory

print('Summary Information %s'%[am for am in dir(summ) if am[0] != '_'])

### Decision Tree

In [ ]:
''' now use the builtin cross-validator '''
#estimator
estim = DecisionTreeClassifier()
# parameters grid
params = (ParamGridBuilder().addGrid(estim.maxBins, [20, 40, 80, 100])\
              .addGrid(estim.maxDepth, [5, 10, 30]).build())
# cross-validator
cv = CrossValidator(estimator=estim, estimatorParamMaps=params, evaluator=acc, numFolds=5)

# run
fitModel = cv.fit(trainASD.select('features', 'label'))

In [ ]:
bestModel = fitModel.bestModel

# best model parameters
print('Max Quant Bins = %0.3f, Max Depth = %0.3f'%(bestModel.getMaxBins(), bestModel.getMaxDepth()))

# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)

# view feature importances
imports = bestModel.featureImportances.toArray()
for (col, imp) in zip(inputColumns, imports):
    print('%s feature importance = %0.3f'%(col, imp))

In [ ]:
# what is available in the model object
print([i for i in dir(bestModel) if i[0] != '_'])

### Random Forest

In [ ]:
''' now use the builtin cross-validator '''
#estimator
estim = RandomForestClassifier(numTrees=20)
# parameters grid
params = (ParamGridBuilder().addGrid(estim.maxBins, [20, 40, 80, 100])\
              .addGrid(estim.maxDepth, [5, 10, 30]).build())
# cross-validator
cv = CrossValidator(estimator=estim, estimatorParamMaps=params, evaluator=acc, numFolds=5)

# run
fitModel = cv.fit(trainASD.select('features', 'label'))

In [ ]:
bestModel = fitModel.bestModel

# best model parameters
print('Max Quant Bins = %0.3f, Max Depth = %0.3f'%(bestModel.getMaxBins(), bestModel.getMaxDepth()))

# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)

# view feature importances
imports = bestModel.featureImportances.toArray()
for (col, imp) in zip(inputColumns, imports):
    print('%s feature importance = %0.3f'%(col, imp))

In [ ]:
# what is available in the model object
print([i for i in dir(bestModel) if i[0] != '_'])

### Now try mlflow

In [ ]:
experiment_name = 'My Experiment'
mlflow.set_experiment(experiment_name='My Experiment')
mlfc = MlflowClient()

experiments = mlfc.list_experiments()

def create_run(expName, mlFlowClient):
    mlflow.set_experiment(experiment_name='My Experiment')
    experiments = mlFlowClient.list_experiments()
    for exp in experiments:
        if expName in exp.name:
            expIndex = experiments.index(exp)
            run = mlFlowClient.create_run(experiments[expIndex].experiment_id)
            print(run)
            return run

In [ ]:
''' use mlflow with the cross validator now '''
run = create_run(experiment_name, mlfc)

#estimator
estim = LogisticRegression()
estimName = 'LogisticReg'
# parameters grid
params = (ParamGridBuilder().addGrid(estim.threshold, [0.4, 0.5, 0.6]).addGrid(estim.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
paramNames = ['threshold', 'elasticnetparam']
# cross-validator
cv = CrossValidator(estimator=logreg, estimatorParamMaps=params, evaluator=acc, numFolds=5)
# run
fitModel = cv.fit(trainASD.select('features', 'label'))

# get results
bestModel = fitModel.bestModel
# best model parameters
print('Threshold = %0.3f, Elastic Net = %0.3f'%(bestModel.getThreshold(), bestModel.getElasticNetParam()))
# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)
# get best parameters
bestParams = bestModel.extractParamMap()

# add stuff to mlflow
mlfc.set_tag(run.info.run_id, 'Estimator', estimName)
mlfc.set_tag(run.info.run_id, 'PRNG Seed', randSeed)
mlfc.set_tag(run.info.run_id, 'Train Perc', trainPerc)
for (key, val) in bestParams.items():
    for parm in paramNames:
        if parm in key.name.lower():
            mlfc.log_param(run.info.run_id, parm, val)
mlfc.log_metric(run.info.run_id, 'Accuracy', testAcc)
mlfc.set_terminated(run.info.run_id)

In [ ]:
sc.stop()